In [16]:
import numpy as np

import sys 
sys.path.append('/home1/efeghhi/ripple_memory/analysis_code/')
from load_data import *
from analyze_data import *
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import decimate

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert

In [17]:
def load_data_np(encoding_mode):
    
    print("Loading data")
    
    region_name = ['HPC']

    condition_on_ca1_ripples = False
    
    if encoding_mode: 
        catFR_dir = '/scratch/efeghhi/catFR1/ENCODING/'
    else:
        catFR_dir = '/scratch/efeghhi/catFR1/IRIonly/'
        
    data_dict = load_data(directory=catFR_dir, region_name=region_name, encoding_mode=encoding_mode)

    if encoding_mode: 
        data_dict = remove_wrong_length_lists(data_dict)
        
    # ca1
    ca1_elecs = [x for x in HPC_labels if 'ca1' in x]
    data_dict_ca1 = select_region(data_dict, ca1_elecs)
    count_num_trials(data_dict_ca1, "ca1")

    data_dict_region = data_dict_ca1
    
    # create clustered int array
    clustered_int = create_semantic_clustered_array(data_dict_region, encoding_mode)
    data_dict_region['clust_int'] = clustered_int

    dd_trials = dict_to_numpy(data_dict_region, order='C')
    
    return dd_trials

In [18]:
dd_trials_recalled = dict(np.load('/home1/efeghhi/ripple_memory/analysis_code/pac_analyses/updated_data/dd_trials_recall.npz'))

In [19]:
dd_trials_encoding = dict(np.load('/home1/efeghhi/ripple_memory/analysis_code/pac_analyses/updated_data/dd_trials_encoding.npz'))

In [32]:
num_diff = 0
for subj in np.unique(dd_trials_recalled['subj']):
    
    encoding_subj_idxs = np.argwhere(dd_trials_encoding['subj']==subj)
    recall_subj_idxs = np.argwhere(dd_trials_recalled['subj']==subj)
    
    enc_elecs = np.unique(dd_trials_encoding['elec_labels'][encoding_subj_idxs])
    rec_elecs = np.unique(dd_trials_recalled['elec_labels'][recall_subj_idxs])
    
    if not np.array_equal(enc_elecs, rec_elecs):
        print("SUBJ: ", subj)
        print("\n")
        print("NOT IN RECALL: ", np.setdiff1d(enc_elecs, rec_elecs))
        print("NOT IN ENCODING: ", np.setdiff1d(rec_elecs, enc_elecs))
        num_diff += 1
        print("\n")
        

SUBJ:  R1065J


NOT IN RECALL:  ['R1065J-1_LE2-LE3' 'R1065J-1_LS1-LS2' 'R1065J-1_LS11-LS12'
 'R1065J-1_LS12-LS13' 'R1065J-1_LS3-LS4' 'R1065J-1_LS4-LS5'
 'R1065J-1_LS5-LS6' 'R1065J-1_LS6-LS7' 'R1065J-1_LS7-LS8']
NOT IN ENCODING:  []


SUBJ:  R1243T


NOT IN RECALL:  ['R1243T-2_LX2-LX3' 'R1243T-2_LX3-LX4']
NOT IN ENCODING:  []


SUBJ:  R1354E


NOT IN RECALL:  ['R1354E-2_1Ld1-1Ld2']
NOT IN ENCODING:  ['R1354E-0_1Ld1-1Ld2']


SUBJ:  R1374T


NOT IN RECALL:  ['R1374T-1_LM3-LM4' 'R1374T-1_LM5-LM6']
NOT IN ENCODING:  []




In [17]:
np.unique(dd_trials_recalled['elec_labels'][recall_subj_idxs], return_counts=True)

(array([], dtype='<U24'), array([], dtype=int64))

In [18]:
encoding_correct_trials = np.argwhere(dd_trials_encoding['correct']==1)
encoding_correct_subj_idxs = np.intersect1d(encoding_correct_trials, encoding_subj_idxs)
electrode_labels_encoding = dd_trials_encoding['elec_labels'][encoding_correct_subj_idxs]
print(np.unique(electrode_labels_encoding, return_counts=True))

(array(['R1313J-0_ROF2-ROF3'], dtype='<U24'), array([78]))


In [19]:
recall_non_clust_trials = np.argwhere(dd_trials_recalled['clust_int']==0)
encoding_non_clust_trials = np.argwhere(dd_trials_encoding['clust_int']==0)
    
recall_non_clust_subj_idxs = np.intersect1d(recall_non_clust_trials, recall_subj_idxs)
encoding_non_clust_subj_idxs = np.intersect1d(encoding_non_clust_trials, encoding_subj_idxs)

electrode_labels_recalled = dd_trials_recalled['elec_labels'][recall_non_clust_subj_idxs]
electrode_labels_encoding = dd_trials_encoding['elec_labels'][encoding_non_clust_subj_idxs]

print("RECALL", np.unique(electrode_labels_recalled, return_counts=True))
print("ENCODING", np.unique(electrode_labels_encoding, return_counts=True))


RECALL (array([], dtype='<U24'), array([], dtype=int64))
ENCODING (array(['R1313J-0_ROF2-ROF3'], dtype='<U24'), array([40]))
